In [1]:
from wetterdienst import Wetterdienst
from wetterdienst.provider.dwd.observation import DwdObservationRequest
import geopandas

In [2]:
DWD = Wetterdienst(provider="dwd", network="observation")
request = DwdObservationRequest(
    parameter=["climate_summary"],
    resolution="daily",
    period="recent"
)

In [3]:
stations = request.all().df
data = request.all().values.all().df

In [4]:
stations = (
    geopandas.GeoDataFrame(
        stations.drop(columns=["longitude", "latitude"]),
        geometry=geopandas.points_from_xy(stations.longitude, stations.latitude),
    )
    .rename(columns={"station_id": "id"})
    .set_index("id")
)
stations


,from_date,to_date,height,name,state,geometry
id,,,,,,
00011,1980-09-01 00:00:00+00:00,2020-09-30 00:00:00+00:00,680.0,Donaueschingen (Landeplatz),Baden-Württemberg,POINT (8.52050 47.97370)
00044,1969-01-01 00:00:00+00:00,2022-07-06 00:00:00+00:00,44.0,Großenkneten,Niedersachsen,POINT (8.23700 52.93360)
00071,1986-11-01 00:00:00+00:00,2019-12-31 00:00:00+00:00,759.0,Albstadt-Badkap,Baden-Württemberg,POINT (8.97840 48.21560)
00073,1959-03-01 00:00:00+00:00,2022-07-06 00:00:00+00:00,340.0,Aldersbach-Kriestorf,Bayern,POINT (13.05060 48.61590)
00078,1961-01-01 00:00:00+00:00,2022-07-06 00:00:00+00:00,65.0,Alfhausen,Niedersachsen,POINT (7.91260 52.48530)
...,...,...,...,...,...,...
15911,2021-06-01 00:00:00+00:00,2022-07-06 00:00:00+00:00,278.0,Großpostwitz-Denkwitz,Sachsen,POINT (14.43430 51.13480)
15976,2020-04-02 00:00:00+00:00,2022-07-06 00:00:00+00:00,216.0,Ober-Olm/Bellem,Rheinland-Pfalz,POINT (8.21070 49.95290)
15978,2020-04-02 00:00:00+00:00,2022-07-06 00:00:00+00:00,316.0,Sembach,Rheinland-Pfalz,POINT (7.86180 49.50440)


In [5]:
stations.to_file("src/assets/stations.geo.json", driver="GeoJSON")

In [6]:
# pivot dataframe
df = data.pivot(index=["station_id", "date"], columns="parameter", values="value")

# drop columns with only NaNs
df = df.dropna(axis=1, how="all")

# convert temperature attributes to celsius
for col in df.columns:
    if col.startswith("temperature"):
        df[col] = df[col].apply(lambda temp_kelvin: temp_kelvin - 273.5)

df

parameter                             cloud_cover_total  humidity  \
station_id date                                                     
00011      2020-02-13 00:00:00+00:00                NaN       NaN   
           2020-02-14 00:00:00+00:00                NaN       NaN   
           2020-02-15 00:00:00+00:00                NaN       NaN   
           2020-02-16 00:00:00+00:00                NaN       NaN   
           2020-02-17 00:00:00+00:00                NaN       NaN   
...                                                 ...       ...   
19172      2022-07-02 00:00:00+00:00                NaN     68.08   
           2022-07-03 00:00:00+00:00                NaN     73.88   
           2022-07-04 00:00:00+00:00                NaN     76.33   
           2022-07-05 00:00:00+00:00                NaN     75.71   
           2022-07-06 00:00:00+00:00                NaN     79.42   

parameter                             precipitation_form  \
station_id date                                            
00011      2020-02-13 00:00:00+00:00                 NaN   
           2020-02-14 00:00:00+00:00                 NaN   
           2020-02-15 00:00:00+00:00                 NaN   
           2020-02-16 00:00:00+00:00                 NaN   
           2020-02-17 00:00:00+00:00                 NaN   
...                                                  ...   
19172      2022-07-02 00:00:00+00:00                 0.0   
           2022-07-03 00:00:00+00:00                 4.0   
           2022-07-04 00:00:00+00:00                 0.0   
           2022-07-05 00:00:00+00:00                 4.0   
           2022-07-06 00:00:00+00:00                 4.0   

parameter                             precipitation_height  pressure_air_site  \
station_id date                                                                 
00011      2020-02-13 00:00:00+00:00                   NaN                NaN   
           2020-02-14 00:00:00+00:00                   NaN                NaN   
           2020-02-15 00:00:00+00:00                   NaN                NaN   
           2020-02-16 00:00:00+00:00                   NaN                NaN   
           2020-02-17 00:00:00+00:00                   NaN                NaN   
...                                                    ...                ...   
19172      2022-07-02 00:00:00+00:00                   0.0                NaN   
           2022-07-03 00:00:00+00:00                   5.5                NaN   
           2022-07-04 00:00:00+00:00                   0.0                NaN   
           2022-07-05 00:00:00+00:00                   0.5                NaN   
           2022-07-06 00:00:00+00:00                  17.4                NaN   

parameter                             pressure_vapor  snow_depth  \
station_id date                                                    
00011      2020-02-13 00:00:00+00:00             NaN         NaN   
           2020-02-14 00:00:00+00:00             NaN         NaN   
           2020-02-15 00:00:00+00:00             NaN         NaN   
           2020-02-16 00:00:00+00:00             NaN         NaN   
           2020-02-17 00:00:00+00:00             NaN         NaN   
...                                              ...         ...   
19172      2022-07-02 00:00:00+00:00          1360.0         NaN   
           2022-07-03 00:00:00+00:00          1530.0         NaN   
           2022-07-04 00:00:00+00:00          1370.0         NaN   
           2022-07-05 00:00:00+00:00          1310.0         NaN   
           2022-07-06 00:00:00+00:00          1330.0         NaN   

parameter                             sunshine_duration  \
station_id date                                           
00011      2020-02-13 00:00:00+00:00                NaN   
           2020-02-14 00:00:00+00:00                NaN   
           2020-02-15 00:00:00+00:00                NaN   
           2020-02-16 00:00:00+00:00                NaN   
           2020-02-17 00:00:

In [7]:
df.to_csv("src/assets/data.csv", float_format="%g", date_format="%Y-%m-%d")